In [9]:
import pandas as pd
import json
import numpy as np

In [10]:
# Get Country Code Dictionary
df = pd.read_csv("../data/raw/owid-co2-data.csv")

country_codes = df.groupby(["iso_code", "country"]).size().reset_index()
country_codes = (
    country_codes[["iso_code", "country"]].set_index("country")["iso_code"].to_dict()
)
json.dumps(country_codes)

# Save to JSON
with open("../data/processed/country_codes.json", "w", encoding="utf-8") as f:
    json.dump(country_codes, f)

In [11]:
# Read in the json file
with open("../data/processed/country_codes.json", encoding="utf-8") as f:
    country_codes = json.load(f)
country_codes

{'Aruba': 'ABW',
 'Afghanistan': 'AFG',
 'Angola': 'AGO',
 'Anguilla': 'AIA',
 'Albania': 'ALB',
 'Andorra': 'AND',
 'United Arab Emirates': 'ARE',
 'Argentina': 'ARG',
 'Armenia': 'ARM',
 'Antarctica': 'ATA',
 'Antigua and Barbuda': 'ATG',
 'Australia': 'AUS',
 'Austria': 'AUT',
 'Azerbaijan': 'AZE',
 'Burundi': 'BDI',
 'Belgium': 'BEL',
 'Benin': 'BEN',
 'Bonaire Sint Eustatius and Saba': 'BES',
 'Burkina Faso': 'BFA',
 'Bangladesh': 'BGD',
 'Bulgaria': 'BGR',
 'Bahrain': 'BHR',
 'Bahamas': 'BHS',
 'Bosnia and Herzegovina': 'BIH',
 'Belarus': 'BLR',
 'Belize': 'BLZ',
 'Bermuda': 'BMU',
 'Bolivia': 'BOL',
 'Brazil': 'BRA',
 'Barbados': 'BRB',
 'Brunei': 'BRN',
 'Bhutan': 'BTN',
 'Botswana': 'BWA',
 'Central African Republic': 'CAF',
 'Canada': 'CAN',
 'Switzerland': 'CHE',
 'Chile': 'CHL',
 'China': 'CHN',
 "Cote d'Ivoire": 'CIV',
 'Cameroon': 'CMR',
 'Democratic Republic of Congo': 'COD',
 'Congo': 'COG',
 'Cook Islands': 'COK',
 'Colombia': 'COL',
 'Comoros': 'COM',
 'Cape Verde': '

### Basic Data Wrangling and Filtering

In [12]:
# Get Country Code Dictionary
df = pd.read_csv("../data/raw/owid-co2-data.csv")

# drop non contry rows (orgnizations like Asia GCP)
reduced_df = df.dropna(subset=['iso_code'])

# filter for year 1900 onwards
reduced_df = reduced_df.query("year >= 1900")

# select used columns
reduced_df = reduced_df[["country", "iso_code", "year",
                 "co2", "temperature_change_from_co2", "co2_per_capita", "co2_per_gdp"]]
# reduced_df.dropna(inplace=True)


### Check for missing values

In [13]:
missing_info = reduced_df.isna().mean() * 100
missing_info = missing_info[missing_info > 0].sort_values(ascending=False)

print("Missing values:")
for column, missing_percentage in missing_info.items():
    print(f"{column}: {df[column].isna().sum()} ({missing_percentage:.2f}%)")

Missing values:
co2_per_gdp: 31768 (53.75%)
co2_per_capita: 21458 (28.73%)
co2: 17750 (27.92%)
temperature_change_from_co2: 6334 (2.16%)


Conclusion: we will only impute the `co2` column since missing values accounts for 27.92% of that column, which will distort the information we want to convey through the dashboard. As for `temperature_change_from_co2`, missing values only accounts for 2.16%, we will leave it as it won't affect the result that much.

### Imputation using backward fill

In [14]:
reduced_df = reduced_df.sort_values(by=['country', 'year'])

# Use groupby on 'country' and apply backward fill within each group for the 'co2' column
imputed_df = reduced_df.groupby('country').apply(lambda group: group.bfill(), include_groups=False).reset_index(drop=True)

In [15]:
missing_info = imputed_df.isna().mean() * 100
missing_info = missing_info[missing_info > 0].sort_values(ascending=False)

print("Missing values:")
for column, missing_percentage in missing_info.items():
    print(f"{column}: {df[column].isna().sum()} ({missing_percentage:.2f}%)")

Missing values:
co2_per_gdp: 31768 (27.54%)
co2_per_capita: 21458 (2.73%)
temperature_change_from_co2: 6334 (2.16%)
co2: 17750 (2.02%)


In [16]:
# save to csv and parquet
reduced_df.to_csv("../data/processed/co2-data.csv", index=False)
reduced_df.to_parquet("../data/processed/co2-data.parquet", compression=None, index=False)